![](img/28_observability.png)

###### <u>Описание критериев наблюдаемости</u>

- **(f-038)** Shauying R.K. *Observability of Nonlinear Systems* $$\varepsilon = \min_i\frac{\Delta_i}{\Delta_{i-1}}$$
    - Важно просто наличие $\varepsilon > 0$ $\to$ достаточное условие наблюдаемости в точке
- **(f-051)** Andrew J. Whalen *Observability and Controllability of Nonlinear Networks The Role of Symmetry* $$\delta(x) = \frac{|\sigma_{min}[O^T O]|}{|\sigma_{max}[O^T O]|}.$$
    - Я выбрал критерий $\delta(x) > 10^{-10}$ (с бухты барахты)

###### <u>Инициализация</u>

In [1]:
from __init__ import *
o1 = kf.init()
o = kf.init(symbolic=True)
t, ω, μ, ρ = o.p.t, o.W_ORB, o.MU, o.RHO 

###### <u>Алгоритм</u> (новое)

In [2]:
def my_diff(expr, power: int = 1, n_c=1, n_d=1):
    """Функция дифференцирует выражение, подставля
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной
    :param n_c: Количество материнских КА
    :param n_d: Количество дочерних КА
    """    
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    global dr_d, dv_d, dq_d, dω_d, dr_c, dv_c, dq_c, dω_c
    if power == 0:
        return expr
    subses, anw = [], (expr if power == 1 else my_diff(expr, power - 1))
    for j in range(3):
        for i in range(n_d):
            subses.extend([(Derivative(r_d[i][j], t), dr_d[i][j]), (Derivative(v_d[i][j], t), dv_d[i][j]), 
                           (Derivative(q_d[i][j+1], t), dq_d[i][j+1]), (Derivative(ω_d[i][j], t), dω_d[i][j])])
        for i in range(n_c):
            subses.extend([(Derivative(r_c[i][j], t), dr_c[i][j]), (Derivative(v_c[i][j], t), dv_c[i][j]),
                           (Derivative(q_c[i][j+1], t), dq_c[i][j+1]), (Derivative(ω_c[i][j], t), dω_c[i][j])])
    anw = anw.diff(t).subs(subses)
    # anw.simplify()
    return anw

def SubRandParams(o, J, n_d: int, n_c: int, n_x: int, n_y: int):
    """Берёт матрицу J размером n_x на n_y, подставляет случайные значения"""
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    # Генерация случайных параметров движения
    s_r = lambda: o.spread(param='r', name="FemtoSat")[0]
    s_v = lambda: o.spread(param='v', name="FemtoSat")[0]
    s_w = lambda: o.spread(param='w', name="FemtoSat")[0]
    rand_params = [(ω, o1.W_ORB), (pi, np.pi), (ρ, o1.RHO), (μ, o1.MU), 
                   (o.V_ORB, o1.V_ORB),  (o.ORBIT_RADIUS, o1.ORBIT_RADIUS), 
                   (o.f.c_resist, o1.f.c_resist), (o.f.mass, o1.f.mass), 
                   (o.c.c_resist, o1.c.c_resist), (o.c.mass, o1.c.mass)] 
    for i in range(3):
        rand_params.append((o.f.size[i], o1.f.size[i]))
        rand_params.append((o.c.size[i], o1.c.size[i]))
        for j in range(3):
            rand_params.append((o.f.J[i, j], o1.f.J[i, j]))
            rand_params.append((o.c.J[i, j], o1.c.J[i, j]))
    for i in range(n_d):
        q = np.random.uniform(-1, 1, 4)
        for j in range(3):
            if i==0:
                _r = [100, 50, 0][j]
                _v = [0, 0.1, 0.1][j]
                _w = [1e-4, 1e-4, 1e-4][j]
                _q = q[j+1] / np.linalg.norm(q)
            else:
                _r, _v, _w, _q = [s_r(), s_v(), s_w(), q[j+1] / np.linalg.norm(q)]
            rand_params.extend([(r_d[i][j], _r), (v_d[i][j], _v), (ω_d[i][j], _w), (q_d[i][j+1], _q)])
    for i in range(n_c):
        q = np.random.uniform(-1, 1, 4)
        for j in range(3):
            if i==0:
                _r = [0, 100, 100][j]
                _v = [0, 0.2, -0.1][j]
                # _w = [1e-5, 1e-5, 1e-5][j]
                _w = 0
                _q = q[j+1] / np.linalg.norm(q)
                # _r, _v, _w, _q = [s_r(), s_v(), s_w(), q[j+1] / np.linalg.norm(q)]
            else:
                _r, _v, _w, _q = [s_r(), s_v(), s_w(), q[j+1] / np.linalg.norm(q)]
            rand_params.extend([(r_c[i][j], _r), (v_c[i][j], _v), (ω_c[i][j], _w), (q_c[i][j+1], _q)])

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)] 
    J_numb = np.array([[float(J[i, j].subs(rand_params).subs(t,0)) for j in range(n_x)] for i in range(n_y)])
    return J_numb

def ShauyingObservabilitySufficientCondition(o, n_d: int, n_c: int, X: list, Y: list, hand_written_deriv: int = None, is_only_xz: bool = False):
    """Проверка достаточного условия наблюдаемости системы. Проверка равномерного отношения миноров матрицы наблюдаемости.
    :param n_d: Количество чипсатов
    :param X: Список неизвестных параметров, которые необходимо найти
    :param Y: Список известных параметров (измерений системы в t₀=0)
    :param my_diff: Функция взятия производной по времени"""
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c
    def print_and_record(report: str, lcl_txt: str):
        print(lcl_txt)
        return report + lcl_txt + "\n"
    report = kf.my_print(f"Количество кубсатов: {n_c}\nКоличество чипсатов: {n_d}\n", if_return=True, bold=True)

    # Количество одномоментных измерений
    l = len(Y)
    # Требуемое количество существующих производных функции измерения
    k = int(len(X) // len(Y)) if hand_written_deriv is None else hand_written_deriv
    txt = f"" if hand_written_deriv is None else f"\033[1mВнимание! Рассчитывается не отношение миноров, а ранг расширенного Якобиана\033[0m\n"
    report = print_and_record(report, txt + f"Неизвестные: n = {len(X)} (на каждый чипсат по {int(len(X) // n_d)} параметров)\nИзвестные: l = {l}\n∃ производные порядка k = {len(X) / len(Y)} (Должна быть целой!)")

    # >>>>>>>> РАССЧЁТ ОТОБРАЖЕНИЯ НАБЛЮДАЕМОСТИ H <<<<<<<< #
    if is_only_xz:
        f = Matrix([Matrix([*dr_d[i][::2], *dv_d[i][::2]]) for i in range(len(dr_d))])
    else:
        f = Matrix([Matrix([*dr_d[i], *dv_d[i]]) for i in range(len(dr_d))])
        # f = Matrix([Matrix([*dr_d[i], *dq_d[i][1:4], *dv_d[i], *dω_d[i]]) for i in range(len(dr_d))]) if o.NAVIGATION_ANGLES else Matrix([Matrix([*dr_d[i], *dv_d[i]]) for i in range(len(dr_d))])
    h, H = Y, []
    for kk in range(k):
        if kk > 0:
            dhdx = Matrix([[h[ll].diff(X[j]) for j in range(len(X))] for ll in range(l)])
            dhdx_f = dhdx @ f
            # dhdx_f.simplify()
            for ll in range(l):
                print(f"__расчёт матрицы H_: k={(kk+1)}/{k}, l={(ll+1)}/{l}")
                h[ll] = my_diff(h[ll], n_c=n_c, n_d=n_d) + dhdx_f[ll]
        H.extend(h)
    H = Matrix(H)
    report = print_and_record(report, f"Размерность матрицы H: {shape(H)}")

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)]
    J = Matrix([[H[i].diff(X[j]) for j in range(len(X))] for i in range(k * l)])
    report = print_and_record(report, f"Размерность матрицы J: {shape(J)}")

    # Подстановка конкретных значений
    J_numb = SubRandParams(o=o, J=J, n_c=n_c, n_d=n_d, n_x=len(X), n_y=k*l)
    _, v, _ = np.linalg.svd(J_numb.T @ J_numb)
    report = print_and_record(report, f"v = {v}")
    report = print_and_record(report, f"σₘₙ/σₘₐₓ = {np.min(v)}/{np.max(v)} = {np.min(v) / np.max(v)} | σ>10⁻⁷: {np.sum(v>1e-7)}/{len(v)} (f-051)")

    # Достаточное условие
    tols = [1e-3, 1e-5, 1e-7, 1e-10, 1e-12, 1e-15]
    txt = f"\nРанг матрицы: {[np.linalg.matrix_rank(J_numb, tol=tol) for tol in tols]} | {tols} (f-055)\n"
    txt += f"Детерминант матрицы: {np.linalg.det(J_numb)}\n" if J_numb.shape[0] == J_numb.shape[1] else ""
    if hand_written_deriv is None:
        A = np.eye(J_numb.shape[0])
    else:
        A = np.repeat(np.eye(len(X)), k, axis=0).reshape(len(X), len(X)*k)[:,:len(Y)*k]
    report = print_and_record(report, txt + f"Следующие параметры не должны быть нулевыми:\n")
    d, Δ, flag, i_min = [], [], True, -1
    print(f"A: {A.shape}, J: {J_numb.shape}")
    for i in range(len(X)):
        tmp = kf.principal_minor(A @ J_numb, i)
        d += [tmp if i == 0 else tmp / Δ[-1]]
        Δ += [tmp]
        report = print_and_record(report, f"Δ_{i} = {d[-1]}" if i == 0 else f"Δ_{i} / Δ_{i-1} = {d[-1]}")
    
        # Чек наблюдаемости
        if flag and abs(d[-1]) < 1e-10:
            i_min = i
            flag = False
        if not flag:
            break

    # Вывод
    if flag:
        txt = f"\n\033[1mВыполнено достаточное условие! Система наблюдаема\033[0m"
    else:
        in_txt = f"Δ_{i_min}" if i_min == 0 else f"Δ_{i_min} / Δ_{i_min-1}"
        txt = f"\n\033[1mНе выполнено достаточное условие. Нулевой параметр: {in_txt} = {d[i_min]}\033[0m"
    # report = print_and_record(report, txt)
    # return H, J, J_numb, Δ, report
    report = print_and_record(report, txt)
    return H, J, J_numb, None, report
    

def observe_system(n_c, n_d, is_d_qw, gains, is_drag, is_only_xz, hand_written_deriv, disp: bool = True, Hcheck: bool = True):
    global r_d, v_d, q_d, ω_d, r_c, v_c, q_c, ω_c, t
    global dr_d, dv_d, dq_d, dω_d, dr_c, dv_c, dq_c, dω_c, dydq
    
    o.GAIN_MODEL_C_N = gains['cN']
    o.GAIN_MODEL_F_N = gains['dN']
    o.c.gain_mode = o.GAIN_MODEL_C = o.GAIN_MODES[gains['cN']]
    o.f.gain_mode = o.GAIN_MODEL_F = o.GAIN_MODES[gains['dN']]
    o.init_choice_params()
    o.NAVIGATION_ANGLES = is_d_qw
    o.DYNAMIC_MODEL['aero drag'] = is_drag
    
    r_d, v_d, q_d, ω_d = get_state_vector(func=kf.get_func, obj='d', n=n_d)
    r_c, v_c, q_c, ω_c = get_state_vector(func=kf.get_func, obj='c', n=n_c)

    o.c.n, o.f.n = n_c, n_d
    o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_brf = r_d, v_d, q_d, ω_d
    o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_brf = r_c, v_c, q_c, ω_c

    x = []
    for i in range(n_d):
        x.extend(r_d[i])
        if is_d_qw:
            x.extend(q_d[i][1:4])
        x.extend(v_d[i])
        if is_d_qw:
            x.extend(ω_d[i])
    x = Matrix(x)
    
    kf.my_print(f"Вектор состояния:", bold=True)
    display(x.T)

    if is_only_xz:
        tmp = []
        for i in range(n_d):
            tmp.extend([r_d[i][0], r_d[i][2], v_d[i][0], v_d[i][2]])
        x = Matrix(tmp)
        kf.my_print(f"Вектор состояния (изменённый):", bold=True)
        display(x.T)

    dr_d, dv_d, dq_d, dω_d = ([0 for _ in range(n_d)] for _ in range(4))
    dr_c, dv_c, dq_c, dω_c = ([0 for _ in range(n_c)] for _ in range(4))
    for i in range(n_d):
        dr_d[i], dv_d[i] = kf.translate_rhs(o=o, obj=o.f, i=i, rv=(r_d[i], v_d[i]), w=ω, mu=μ, rho=ρ)
        dq_d[i], dω_d[i] = kf.attitude_rhs(o=o, obj=o.f, t=t, i=i, qw=(q_d[i], ω_d[i]))
    for i in range(n_c):
        dr_c[i], dv_c[i] = kf.translate_rhs(o=o, obj=o.c, i=i, rv=(r_c[i], v_c[i]), w=ω, mu=μ, rho=ρ)
        dq_c[i], dω_c[i] = kf.attitude_rhs(o=o, obj=o.c, t=t, i=i, qw=(q_c[i], ω_c[i]))
    if disp:
        kf.my_print(f"Динамика: \n{' '*10}dr = ", bold=True)
        display(dr_d[0])
        kf.my_print(f"{' '*10}dv = ", bold=True)
        display(dv_d[0])
        kf.my_print(f"{' '*10}dq = ", bold=True)
        display(dq_d[0])
        # kf.my_print(f"{' '*10}dω = ", bold=True)
    
    y = kf.measure_antennas_power(c=o.c, f=o.f, vrs=o, p=o.p, j=int(len(x) // n_d), state=x)
    # kf.my_print(f"Вектор измерени:", bold=True)
    # display(y.T)

    # >>>>>>>>>>>>>>>>> Проверка матрицы H <<<<<<<<<<<<<<<<<
    if Hcheck:
        H_1 = kf.h_matrix(t=t, o=o, f=o.f, c=o.c, r_f=r_d, r_c=r_c, 
                          q_f=[Matrix(q_d[i][1:4]) for i in range(o.f.n)], 
                          q_c=[Matrix(q_c[i][1:4]) for i in range(o.c.n)])
        H_2 = zeros(*H_1.shape)
        for ix, xx in enumerate(x):
            for iy, yy in enumerate(y):
                H_2[iy, ix] = yy.diff(xx)
        kf.my_print("Проверка матрицы H:", bold=True)
        #tmp = H_1 - H_2
        #tmp.simplify()
        #display(tmp)
        H_12 = SubRandParams(o=o, J=H_1 - H_2, n_d=o.f.n, n_c=o.c.n, n_x=len(x), n_y=len(y))
        print(H_12)
    
    return ShauyingObservabilitySufficientCondition(o=o, n_c=n_c, n_d=n_d, X=x, Y=y, hand_written_deriv=hand_written_deriv, is_only_xz=is_only_xz)

###### <u>Case №1</u>: No?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=3, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №2</u>: Yes?

In [3]:
H, J, Jn, Δ, report = observe_system(n_c=2, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор состояния (изменённый):


Matrix([[r_0^d_x(t), r_0^d_z(t), v_0^d_x(t), v_0^d_z(t)]])

Количество кубсатов: 2
Количество чипсатов: 1

Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 2
∃ производные порядка k = 2.0 (Должна быть целой!)
__расчёт матрицы H_: k=2/2, l=1/2
__расчёт матрицы H_: k=2/2, l=2/2
Размерность матрицы H: (4, 1)
Размерность матрицы J: (4, 4)
v = [5.11389216e-12 1.62658057e-12 1.39505775e-13 1.45197334e-15]
σₘₙ/σₘₐₓ = 1.451973336613137e-15/5.11389216243964e-12 = 0.00028392724963532604 | σ>10⁻⁷: 0/4 (f-051)

Ранг матрицы: [0, 0, 3, 4, 4, 4] | [0.001, 1e-05, 1e-07, 1e-10, 1e-12, 1e-15] (f-055)
Детерминант матрицы: 4.10477149875247e-26
Следующие параметры не должны быть нулевыми:

A: (4, 4), J: (4, 4)
Δ_0 = -2.2222222222222222e-07
Δ_1 / Δ_0 = 4.547719121953598e-07
Δ_2 / Δ_1 = -4.446656813560581e-07
Δ_3 / Δ_2 = 9.134278599125683e-07

Выполнено достаточное условие! Система наблюдаема


###### <u>Case №3</u>: Yes?

In [16]:
H, J, Jn, Δ, report = observe_system(n_c=3, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Количество кубсатов: 3
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 3
∃ производные порядка k = 2.0 (Должна быть целой!)
__расчёт матрицы H_: k=2/2, l=1/3
__расчёт матрицы H_: k=2/2, l=2/3
__расчёт матрицы H_: k=2/2, l=3/3
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
-1/781250
-2.02401037840931e-7
-5.07820042868473e-7
4.09600000000000e-9
5.10819720811414e-10
1.81990123773564e-9
-1/1562500
-1.67534712127574e-7
-1.42280075637396e-7
-5.12000000000000e-10
8.12118486310385e-11
-2.15225745471797e-10
0
2.16298674431748e-8
-2.09991324983052e-7
-2.56000000000000e-9
-3.17754288277653e-10
-1.72627776369641e-10
0
0
0
-1/390625
-4.04802075681861e-7
-1.01564008573695e-6
0
0
0
-1/781250
-3.35069424255149e-7
-2.84560151274793e-7
0
0
0
0
4.32597348863496e-8
-4.19982649966105e-7
v = [9.54726993e-12 2.38680519e-12 2.04534103e-13 5.11332322e-14
 7.12030092e-15 1.78000739e-15]
σₘₙ/σₘₐₓ = 1.780007392206537e-15/9.547269930611317e-12 = 0.000186441

###### <u>Case №4</u>: Yes?

In [17]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=True, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Вектор состояния (изменённый):


Matrix([[r_0^d_x(t), r_0^d_z(t), v_0^d_x(t), v_0^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 1
∃ производные порядка k = 4.0 (Должна быть целой!)
__расчёт матрицы H_: k=2/4, l=1/1
__расчёт матрицы H_: k=3/4, l=1/1
__расчёт матрицы H_: k=4/4, l=1/1



KeyboardInterrupt



###### <u>Case №5</u>: Yes?

In [19]:
H, J, Jn, Δ, report = observe_system(n_c=2, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 0, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Количество кубсатов: 2
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 2
∃ производные порядка k = 3.0 (Должна быть целой!)
__расчёт матрицы H_: k=2/3, l=1/2
__расчёт матрицы H_: k=2/3, l=2/2
__расчёт матрицы H_: k=3/3, l=1/2
__расчёт матрицы H_: k=3/3, l=2/2
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
v = [9.38210095e-12 2.52871128e-12 2.34542888e-12 6.32151715e-13
 3.20477050e-17 4.95741043e-24]
σₘₙ/σₘₐₓ = 4.957410433988719e-24/9.382100947973991e-12 = 5.283902253321248e-13 | σ>10⁻⁷: 0/6 (f-051)

Ранг матрицы: [0, 0, 4, 5, 6, 6] | [0.001, 1e-05, 1e-07, 1e-10, 1e-12, 1e-15] (f-055)
Детерминант матрицы: -7.475735192465199e-44
Следующие параметры не должны быть нулевыми:

A: (6, 6), J: (6, 6)
Δ_0 = -1.2800000000000009e-06
Δ_1 / Δ_0 = 6.765782845027379e-07
Δ_2 / Δ_1 = -4.823177747684256e-09
Δ_3 / Δ_2 = -1.749313940936292e-07
Δ_4 / Δ_3 = -1.2064307115619648e-08
Δ_5 / Δ_4 = -8.48051987674479e-09

Выполнено достаточное условие! Сис

###### <u>Case №6</u>: Yes?

In [36]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=False, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

Вектор состояния:


Matrix([[r_0^d_x(t), r_0^d_y(t), r_0^d_z(t), v_0^d_x(t), v_0^d_y(t), v_0^d_z(t)]])

Количество кубсатов: 1
Количество чипсатов: 1

Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 3
∃ производные порядка k = 2.0 (Должна быть целой!)
__расчёт матрицы H_: k=2/2, l=1/3
__расчёт матрицы H_: k=2/2, l=2/3
__расчёт матрицы H_: k=2/2, l=3/3
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)
v = [1.55370856e-12 7.80379760e-13 3.88420212e-13 2.78723826e-13
 1.95091013e-13 6.96796180e-14]
σₘₙ/σₘₐₓ = 6.967961797813558e-14/1.5537085550107615e-12 = 0.044847289894502096 | σ>10⁻⁷: 0/6 (f-051)

Ранг матрицы: [0, 0, 6, 6, 6, 6] | [0.001, 1e-05, 1e-07, 1e-10, 1e-12, 1e-15] (f-055)
Детерминант матрицы: 4.2242269911857216e-38
Следующие параметры не должны быть нулевыми:

A: (6, 6), J: (6, 6)
Δ_0 = -4.0897448148144277e-07
Δ_1 / Δ_0 = 4.2461325724584713e-07
Δ_2 / Δ_1 = 4.1844546947364486e-07
Δ_3 / Δ_2 = -8.179489629628842e-07
Δ_4 / Δ_3 = 8.492265144916988e-07
Δ_5 / Δ_4 = 8.368909389472913e-07

Выполнено достаточное условие! Система наблюдаема


###### <u>Case №7</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'dN': 0},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №8</u>: No?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'dN': 1},
                                     hand_written_deriv=None, disp=False, Hcheck=False)

###### <u>Case №9</u>: Yes?

In [ ]:
H, J, Jn, Δ, report = observe_system(n_c=1, n_d=1, 
                                     is_d_qw=False, 
                                     is_drag=True, 
                                     is_only_xz=False, 
                                     gains={'cN': 3, 'dN': 1},
                                     hand_written_deriv=None, disp=False, Hcheck=False)